In [323]:
# Импорт библиотек
import pandas as pd
from datetime import datetime
from ipywidgets import interact, widgets, Layout

In [324]:
def df_to_hist(df):
    with open("hist.txt", 'w') as file:
        file.write("EFORM WELL 'DD.MM.YYYY' MDL MDU SKIN MULT rad\n\nETAB\n")
    wellnames = df_object['Скважина'].unique().tolist()
    for well in wellnames:
        selected_well = df.loc[df["Скважина"] == well]
        selected_well.to_csv('hist.txt', mode = 'a', index = False, header = False, sep= " ")
        with open("hist.txt", 'a') as file:
            file.write('\n')
    with open("hist.txt", 'a') as file:
       file.write('ENDE')

In [325]:
# Подготовка данных
df = pd.read_excel("перфорации.xlsx") # считываем эксель-файл
filter = df['Пласт'].unique().tolist()
w = widgets.SelectMultiple(options = filter )
w

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


SelectMultiple(options=('ЮВ1/1', 'БВ6-7', 'ПК', 'БВ7', nan), value=())

In [326]:
df = df.loc[df['Пласт'].isin(w.value)]
df = df.drop(columns = ['ГИС','Перфоратор','N отв','Hнач(abs)','Hкон(abs)','Hнач(abs,тр)','Hкон(abs,тр)','Плотность']) # удаляю ненужные строки из выгрузки
change_perf = {'ПЕРФОРАЦИЯ':'PERF',"ЗАЛИВКА":'SQUE','ПЕРВИЧ.ПЕРФ.':'PERF','ПЕРЕСТРЕЛ':'PERF','ПРИОБЩЕНИЕ':'PERF','ДОСТРЕЛ':'PERF'}
df_object = df
df_object['Дата'] = df_object['Дата'].dt.to_period('M').dt.to_timestamp() # заменяю дату на первое число месяца
df_object = df_object.sort_values(['Скважина','Дата'], ascending=[True, True])
df_object['Дата'] = pd.to_datetime (df_object['Дата']).apply(lambda x: x.strftime('%d.%m.%y')) # преобразую формат даты в нужный вид
df_object['Hнач(md)'],df_object['Hкон(md)']= df_object['Hнач(md)'].astype(int),df_object['Hкон(md)'].astype(int)
df_object

,Скважина,Дата,Пласт,Тип,Hнач(md),Hкон(md),Открыт
0,1006,01.05.23,ЮВ1/1,ЗАЛИВКА,3376,3379,Нет
1,1006,01.05.23,ЮВ1/1,ЗАЛИВКА,3357,3370,Нет
2,1006,01.07.23,БВ6-7,ПЕРВИЧ.ПЕРФ.,2849,2857,Да
3,1008,01.09.23,БВ6-7,ПЕРВИЧ.ПЕРФ.,3067,3814,Да
4,1008,01.09.23,БВ6-7,ПЕРВИЧ.ПЕРФ.,2810,2917,Да
...,...,...,...,...,...,...,...
219,890,01.06.15,ЮВ1/1,ЗАЛИВКА,2846,2857,Нет
222,911,01.04.89,ЮВ1/1,ЗАЛИВКА,2963,2974,Нет
224,911,01.04.23,БВ6-7,ПЕРВИЧ.ПЕРФ.,2547,2556,Да
225,913,01.09.89,ЮВ1/1,ЗАЛИВКА,2902,2909,Нет


In [327]:
# ВВОД значений
skin = 0
mult = 1
radius = 0.084
df_object['skin'] = skin
df_object['mult'] = float(mult)
df_object['radius'] = float(radius)
df_object["Открытость"] = '--Открыт:'+ df_object["Открыт"]
df_object["Пласт2"] = df_object['Пласт']
df_object["Тип2"] = df_object["Тип"]
df_object = df_object.replace({"Тип": change_perf})
df_object = df_object.drop(columns = ['Открыт','Пласт'])
df_object


,Скважина,Дата,Тип,Hнач(md),Hкон(md),skin,mult,radius,Открытость,Пласт2,Тип2
0,1006,01.05.23,SQUE,3376,3379,0,1.0,0.084,--Открыт:Нет,ЮВ1/1,ЗАЛИВКА
1,1006,01.05.23,SQUE,3357,3370,0,1.0,0.084,--Открыт:Нет,ЮВ1/1,ЗАЛИВКА
2,1006,01.07.23,PERF,2849,2857,0,1.0,0.084,--Открыт:Да,БВ6-7,ПЕРВИЧ.ПЕРФ.
3,1008,01.09.23,PERF,3067,3814,0,1.0,0.084,--Открыт:Да,БВ6-7,ПЕРВИЧ.ПЕРФ.
4,1008,01.09.23,PERF,2810,2917,0,1.0,0.084,--Открыт:Да,БВ6-7,ПЕРВИЧ.ПЕРФ.
...,...,...,...,...,...,...,...,...,...,...,...
219,890,01.06.15,SQUE,2846,2857,0,1.0,0.084,--Открыт:Нет,ЮВ1/1,ЗАЛИВКА
222,911,01.04.89,SQUE,2963,2974,0,1.0,0.084,--Открыт:Нет,ЮВ1/1,ЗАЛИВКА
224,911,01.04.23,PERF,2547,2556,0,1.0,0.084,--Открыт:Да,БВ6-7,ПЕРВИЧ.ПЕРФ.
225,913,01.09.89,SQUE,2902,2909,0,1.0,0.084,--Открыт:Нет,ЮВ1/1,ЗАЛИВКА


In [328]:
df_to_hist(df_object)